In [ ]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 3.3 MB/s 
     |████████████████████████████████| 6.5 MB 39.2 MB/s 
     |████████████████████████████████| 596 kB 60.6 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 895 kB 61.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import BertTokenizer, BertModel
import torch.optim as optim
import pandas as pd
import numpy as np

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

30522

In [ ]:
max_len = 128

In [ ]:
def trim_sentence(sent):
  curr_len = len(sent)
  
  sent = sent[:min(max_len, curr_len)]
  return sent

In [ ]:
def pad_sentence(sent, max_len):

  padding = [0 for _ in range(len(max_len-len(sent)))]
  padded_sent = sent.extend(padding)

  return padded_sent

In [ ]:
def get_attention(sent):

  attention = [1 for _ in range(len(sent))]
  return attention

In [ ]:
def process_sent1(sent):
  return tokenizer.cls_token + " " + sent + " " + tokenizer.sep_token

def process_sent2(sent):
  return sent + " " + tokenizer.sep_token

In [ ]:
def get_sent1_token_id(sent):
  return [0]*len(sent)

def get_sent2_token_id(sent):
  return [1]*len(sent)

In [ ]:
def join_mask(mask):

  mask = [str(i) for i in mask]
  mask = " ".join(mask)

  return mask

In [ ]:
def tokenize_sent(sent):

  tokens = tokenizer.tokenize(sent)
  return tokens

In [ ]:
def join_seq(sent):
  return " ".join(sent)

def join_mask(sent):
  sent = [int(val) for val in sent]
  return " ".join(sent)

In [ ]:
def get_bert_format(df):
  for i in range(len(df)):

    sent1 = df["sentence1"]
    sent2 = df["sentence2"]
    
    sent1 = sent1.apply(trim_sentence)
    sent2 = sent2.apply(trim_sentence)

    sent1 = sent1.apply(process_sent1)
    sent2 = sent2.apply(process_sent2)

    tokens1 = sent1.apply(tokenize_sent)
    tokens2 = sent2.apply(tokenize_sent)
    
    df["token_ids1"] = get_sent1_token_id(tokens1)
    df["token_ids2"] = get_sent2_token_id(tokens2)
    df["token_ids"] = df["token_ids1"] + df["token_ids2"]

    df["tmp_tokens"] = tokens1+tokens2
    df["attention_sent"] = df["tmp_tokens"].apply(get_attention)

    df["tokens"] = df["tmp_tokens"].apply(join_seq)

    df["attention_sent"] = df["attention_sent"].apply(join_mask)
    df["token_ids"] = df["token_ids"].apply(join_mask)

    return df["tokens"], df["attention_sent"], df["token_ids"]

In [ ]:
train_data = pd.read_csv("snli_1.0_train.txt", sep="\t")
train_data = train_data[["sentence1", "sentence2", "gold_label"]]

train_data["tokens"], train_data["attention_sent"], train_data["token_ids"] = get_bert_format(train_data)
train_data = train_data[["tokens", "attention_sent", "token_ids", "gold_label"]]

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
class NLIModel(nn.Module):

  def __init__(self, model, hidden_neurons, output_neurons):
    super().__init__()
    self.model = model
    feature_vec = model.config.to_dict()['hidden_size']
    self.dense = nn.Linear(feature_vec, output_neurons)

  def forward(self, tokens, attention_mask, token_type):

    x = self.model(input_ids = tokens, attention_mask = attention_mask, token_type_ids= token_type)[1]
    y = self.dense(x)

    return y

In [ ]:
nli_model = NLIModel(bert_model, 1024, 3)
optim = AdamW(nli_model.parameters(), lr=2e-5, eps=1e-6, correct_bias=False)
loss_fnc = nn.CrossEntropyLoss()

In [ ]:
def predict()